# Instalar e Configurar o MLflow
Instale o MLflow usando pip e configure o ambiente para iniciar o servidor de rastreamento (tracking server).

In [1]:
# Instalar o MLflow
!pip install mlflow


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\mpatu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Iniciar o servidor de rastreamento
!mlflow server --host 127.0.0.1 --port 9080

^C


In [3]:
# Configurar o ambiente para iniciar o servidor de rastre amento
import os

# Definir o diretório onde os artefatos serão armazenados
os.environ["MLFLOW_TRACKING_URI"] = "file:./mlruns"

# Registrar Experimentos e Executar Treinamentos
Crie experimentos no MLflow, registre métricas e parâmetros durante o treinamento de um modelo simples.

In [4]:
# Importar bibliotecas necessárias
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Criar um experimento no MLflow
mlflow.set_experiment("Experimento_RandomForest")

# Gerar dados de exemplo
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



2025/03/29 06:19:54 INFO mlflow.tracking.fluent: Experiment with name 'Experimento_RandomForest' does not exist. Creating a new experiment.


In [5]:
# Iniciar uma nova execução no MLflow
with mlflow.start_run():
    # Definir hiperparâmetros do modelo
    n_estimators = 100
    max_depth = 5
    random_state = 42

    # Registrar hiperparâmetros no MLflow
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Treinar o modelo
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    # Fazer previsões e calcular métricas
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Registrar métricas no MLflow
    mlflow.log_metric("mse", mse)

    # Registrar o modelo treinado no MLflow
    mlflow.sklearn.log_model(model, "random_forest_model")

    print(f"Modelo treinado e registrado com MSE: {mse}")

2025/03/29 06:19:55 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2025/03/29 06:20:00 WARNING mlflow.models.model: Model logged without a signature and input example. Ple

Modelo treinado e registrado com MSE: 4349.278786652099


# Versionamento de Dados
Demonstre como registrar e versionar datasets usados no treinamento utilizando o MLflow.

In [8]:
# Registrar e versionar datasets no MLflow

import pandas as pd
from sklearn.datasets import make_regression

# Gerar um dataset de exemplo
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
dataset = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
dataset["target"] = y

# Salvar o dataset em um arquivo CSV
dataset_path = "dataset.csv"
dataset.to_csv(dataset_path, index=False)

# Iniciar uma nova execução no MLflow para registrar o dataset
with mlflow.start_run():
    # Registrar o dataset como um artefato no MLflow
    mlflow.log_artifact(dataset_path, artifact_path="datasets")

    print(f"Dataset registrado no MLflow: {dataset_path}")

Dataset registrado no MLflow: dataset.csv


# Versionamento de Modelos
Mostre como salvar e versionar modelos treinados no repositório do MLflow.

In [9]:
# Registrar e versionar modelos no MLflow

# Importar bibliotecas necessárias
from mlflow.tracking import MlflowClient

# Inicializar o cliente do MLflow
client = MlflowClient()

# Obter o ID do experimento atual
experiment_id = mlflow.get_experiment_by_name("Experimento_RandomForest").experiment_id

# Obter a última execução do experimento
runs = client.search_runs(experiment_ids=experiment_id, order_by=["start_time desc"], max_results=1)
run_id = runs[0].info.run_id

# Registrar o modelo treinado como um modelo versionado no MLflow
model_uri = f"runs:/{run_id}/random_forest_model"
registered_model_name = "RandomForestModel"

# Registrar o modelo
mlflow.register_model(model_uri=model_uri, name=registered_model_name)

print(f"Modelo registrado como '{registered_model_name}' com sucesso!")

# Listar versões do modelo registrado
versions = client.get_latest_versions(name=registered_model_name)
for version in versions:
    print(f"Modelo: {version.name}, Versão: {version.version}, Status: {version.current_stage}")

Modelo registrado como 'RandomForestModel' com sucesso!
Modelo: RandomForestModel, Versão: 1, Status: None


Successfully registered model 'RandomForestModel'.
Created version '1' of model 'RandomForestModel'.
C:\Users\mpatu\AppData\Local\Temp\ipykernel_35152\2450743903.py:26: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(name=registered_model_name)


# Realizar o Deploy de um Modelo
Demonstre como realizar o deploy de um modelo registrado usando o MLflow para servir previsões em tempo real.

In [10]:
# Realizar o Deploy de um Modelo

# Importar bibliotecas necessárias
from mlflow.pyfunc import load_model
import pandas as pd

# Definir o nome do modelo registrado
registered_model_name = "RandomForestModel"

# Carregar o modelo registrado na última versão
model_uri = f"models:/{registered_model_name}/Production"
model = load_model(model_uri)

# Criar um novo conjunto de dados para teste
X_new, _ = make_regression(n_samples=5, n_features=10, noise=0.1, random_state=42)
X_new_df = pd.DataFrame(X_new, columns=[f"feature_{i}" for i in range(X_new.shape[1])])

# Fazer previsões usando o modelo carregado
predictions = model.predict(X_new_df)

# Exibir as previsões
print("Previsões realizadas pelo modelo em produção:")
print(predictions)

C:\Users\mpatu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mlflow\store\artifact\utils\models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


MlflowException: No versions of model with name 'RandomForestModel' and stage 'Production' found